# Reading Level Classifier

## Imports

In [158]:
import pandas as pd
import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import pandas as pd
import nltk
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
import matplotlib.pyplot as plt
import string
import re
import numpy as np

In [159]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /Users/javm/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [160]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

## Data Cleaning

### New Feats

In [162]:
new_feat_df = pd.read_csv('/Users/javm/Desktop/Flatiron4/allfeatures-ose-final.csv')
new_feat_df

,fileName,AoA_Bird_Lem,AoA_Bristol_Lem,AoA_Cort_Lem,AoA_Kup,AoA_Kup_Lem,DISC_RefExprDefArtPerSen,DISC_RefExprDefArtPerWord,DISC_RefExprPerProPerWord,DISC_RefExprPerPronounsPerSen,...,PROPORTION_INDEF_NP_REFCHAIN,PROPORTION_DEF_NP_REFCHAIN,PROPORTION_PERSONAL_PRONOUNS_REFCHAIN,PROPORTION_POSSESIVE_DETERMINERS_REFCHAIN,PROPORTION_DEMONSTRATIVE_DETERMINERS_REFCHAIN,PROPORTION_DEMONSTRATIVE_PRONOUNS_REFCHAIN,PROPORTION_REFLEXIVE_PRONOUNS_REFCHAIN,PROPORTION_PROPER_NOUNS_REFCHAIN,AVG_LEN_REFCHAIN,readinglevel
0,Amazon-adv.txt,3.36,1.50,2.44,4.67,6.10,1.25,0.07,0.01,0.22,...,0.200000,0.485714,0.200000,0.0,0.0,0.0,0.000000,0.171429,2.500000,adv
1,Amazon-ele.txt,3.65,1.11,2.43,4.56,5.76,0.84,0.06,0.02,0.27,...,0.115385,0.615385,0.230769,0.0,0.0,0.0,0.000000,0.461538,2.600000,ele
2,Amazon-int.txt,3.51,1.30,2.43,4.60,5.93,1.12,0.07,0.01,0.25,...,0.166667,0.466667,0.233333,0.0,0.0,0.0,0.000000,0.233333,2.727273,int
3,Amsterdam-adv.txt,3.77,1.35,2.57,4.43,5.94,1.31,0.05,0.03,0.89,...,0.179487,0.461538,0.358974,0.0,0.0,0.0,0.000000,0.179487,3.545455,adv
4,Amsterdam-ele.txt,3.82,1.17,2.74,4.36,5.51,1.13,0.05,0.04,0.90,...,0.208333,0.375000,0.416667,0.0,0.0,0.0,0.000000,0.166667,3.692308,ele
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562,Zero-Hours-ele.txt,3.98,1.48,2.55,4.47,5.98,0.95,0.04,0.03,0.60,...,0.195122,0.268293,0.317073,0.0,0.0,0.0,0.024390,0.048780,3.727273,ele
563,Zero-Hours-int.txt,4.06,1.40,2.52,4.30,5.98,1.46,0.05,0.02,0.61,...,0.170213,0.382979,0.255319,0.0,0.0,0.0,0.021277,0.021277,3.615385,int
564,climate-change--adv.txt,3.81,1.41,2.45,4.33,5.91,1.16,0.05,0.01,0.40,...,0.185185,0.629630,0.333333,0.0,0.0,0.0,0.000000,0.222222,2.700000,adv
565,climate-change--ele.txt,4.16,1.28,2.68,3.99,5.40,0.96,0.05,0.01,0.27,...,0.320000,0.480000,0.320000,0.0,0.0,0.0,0.000000,0.200000,2.777778,ele


In [163]:
new_feat_df.rename(columns = {'readinglevel':'Level'}, inplace = True)

In [164]:
new_feat_df.rename(columns = {'fileName':'Title'}, inplace = True)

In [165]:
new_feat_df['Level'].replace(['adv', 'int', 'ele'],
                        [2,1,0], inplace=True)

In [166]:
new_feat_df

,Title,AoA_Bird_Lem,AoA_Bristol_Lem,AoA_Cort_Lem,AoA_Kup,AoA_Kup_Lem,DISC_RefExprDefArtPerSen,DISC_RefExprDefArtPerWord,DISC_RefExprPerProPerWord,DISC_RefExprPerPronounsPerSen,...,PROPORTION_INDEF_NP_REFCHAIN,PROPORTION_DEF_NP_REFCHAIN,PROPORTION_PERSONAL_PRONOUNS_REFCHAIN,PROPORTION_POSSESIVE_DETERMINERS_REFCHAIN,PROPORTION_DEMONSTRATIVE_DETERMINERS_REFCHAIN,PROPORTION_DEMONSTRATIVE_PRONOUNS_REFCHAIN,PROPORTION_REFLEXIVE_PRONOUNS_REFCHAIN,PROPORTION_PROPER_NOUNS_REFCHAIN,AVG_LEN_REFCHAIN,Level
0,Amazon-adv.txt,3.36,1.50,2.44,4.67,6.10,1.25,0.07,0.01,0.22,...,0.200000,0.485714,0.200000,0.0,0.0,0.0,0.000000,0.171429,2.500000,2
1,Amazon-ele.txt,3.65,1.11,2.43,4.56,5.76,0.84,0.06,0.02,0.27,...,0.115385,0.615385,0.230769,0.0,0.0,0.0,0.000000,0.461538,2.600000,0
2,Amazon-int.txt,3.51,1.30,2.43,4.60,5.93,1.12,0.07,0.01,0.25,...,0.166667,0.466667,0.233333,0.0,0.0,0.0,0.000000,0.233333,2.727273,1
3,Amsterdam-adv.txt,3.77,1.35,2.57,4.43,5.94,1.31,0.05,0.03,0.89,...,0.179487,0.461538,0.358974,0.0,0.0,0.0,0.000000,0.179487,3.545455,2
4,Amsterdam-ele.txt,3.82,1.17,2.74,4.36,5.51,1.13,0.05,0.04,0.90,...,0.208333,0.375000,0.416667,0.0,0.0,0.0,0.000000,0.166667,3.692308,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562,Zero-Hours-ele.txt,3.98,1.48,2.55,4.47,5.98,0.95,0.04,0.03,0.60,...,0.195122,0.268293,0.317073,0.0,0.0,0.0,0.024390,0.048780,3.727273,0
563,Zero-Hours-int.txt,4.06,1.40,2.52,4.30,5.98,1.46,0.05,0.02,0.61,...,0.170213,0.382979,0.255319,0.0,0.0,0.0,0.021277,0.021277,3.615385,1
564,climate-change--adv.txt,3.81,1.41,2.45,4.33,5.91,1.16,0.05,0.01,0.40,...,0.185185,0.629630,0.333333,0.0,0.0,0.0,0.000000,0.222222,2.700000,2
565,climate-change--ele.txt,4.16,1.28,2.68,3.99,5.40,0.96,0.05,0.01,0.27,...,0.320000,0.480000,0.320000,0.0,0.0,0.0,0.000000,0.200000,2.777778,0


### Advanced List

In [167]:
adv_list = []
advanced = '/Users/javm/Desktop/Flatiron4/Reading-Level-Classifier/Texts-SeparatedByReadingLevel/Adv-Txt'
for i in (os.listdir(advanced)):
    with open(advanced+'/'+i) as f:
        lines = f.read()
        adv_list.append(lines)
        
        

In [168]:
adv_list_title = list(os.listdir(advanced))

In [169]:
# initialize list of lists
data1 = adv_list
# Create the pandas DataFrame
adv_df = pd.DataFrame(data1, columns = ['Text'] )
# print dataframe.
adv_df['Level'] = 2
adv_df

,Text,Level
0,﻿It was not so much how hard people found the ...,2
1,﻿Swedish prisons have long had a reputation ar...,2
2,A lonely old man living in a crater on the moo...,2
3,﻿Facebook has lost millions of users per month...,2
4,﻿Noise emanating from passing ships may distur...,2
...,...,...
184,﻿Scientists have created an “atlas of the brai...,2
185,﻿The forests – and suburbs – of Europe are ech...,2
186,"﻿James Hamblin, senior editor of American maga...",2
187,﻿Prince Harry has flown out of Afghanistan at ...,2


In [170]:
data11 = adv_list_title
# Create the pandas DataFrame
adv_title_df = pd.DataFrame(data11, columns = ['Title'] )
# print dataframe.
adv_title_df

,Title
0,WNL Shocking-adv.txt
1,Swedish prisons-adv.txt
2,WNL John Lewis-adv.txt
3,Facebook deserted by millions of users-adv.txt
4,WNL Ships noise-adv.txt
...,...
184,WNL Neuroscientists-adv.txt
185,WNL Brown bears-adv.txt
186,WNL Why we should -adv.txt
187,Prince Harry-adv.txt


In [171]:
adv_df1 = pd.concat([adv_title_df,adv_df], axis = 1)
adv_df1

,Title,Text,Level
0,WNL Shocking-adv.txt,﻿It was not so much how hard people found the ...,2
1,Swedish prisons-adv.txt,﻿Swedish prisons have long had a reputation ar...,2
2,WNL John Lewis-adv.txt,A lonely old man living in a crater on the moo...,2
3,Facebook deserted by millions of users-adv.txt,﻿Facebook has lost millions of users per month...,2
4,WNL Ships noise-adv.txt,﻿Noise emanating from passing ships may distur...,2
...,...,...,...
184,WNL Neuroscientists-adv.txt,﻿Scientists have created an “atlas of the brai...,2
185,WNL Brown bears-adv.txt,﻿The forests – and suburbs – of Europe are ech...,2
186,WNL Why we should -adv.txt,"﻿James Hamblin, senior editor of American maga...",2
187,Prince Harry-adv.txt,﻿Prince Harry has flown out of Afghanistan at ...,2


### Intermediate List

In [172]:
int_list = []
intermediate = '/Users/javm/Desktop/Flatiron4/Reading-Level-Classifier/Texts-SeparatedByReadingLevel/Int-Txt'
for i in (os.listdir(intermediate)):
    with open(intermediate+'/'+i) as f:
        lines = f.read()
        int_list.append(lines)
        
#print(int_list)

In [173]:
int_list_title = list(os.listdir(intermediate))

In [174]:
# initialize list of lists
data3 = int_list
# Create the pandas DataFrame
int_df = pd.DataFrame(data3, columns = ['Text'] )
# print dataframe.
int_df['Level'] = 1
int_df

,Text,Level
0,Intermediate \nValdevaqueros is one of the las...,1
1,Intermediate \nThe Duchess of Cambridge gave b...,1
2,Intermediate \nCathal Redmond was swimming off...,1
3,Intermediate \nIt has mapped the worlds highes...,1
4,Intermediate \nThe customer next to you in the...,1
...,...,...
184,Intermediate \nEver since he was diagnosed HIV...,1
185,Intermediate \nGalina Zaglumyonova was woken i...,1
186,"Intermediate \nAll six numbers match, so its t...",1
187,"Intermediate \nAccording to a recent report, t...",1


In [175]:
data22 = int_list_title
# Create the pandas DataFrame
int_title_df = pd.DataFrame(data22, columns = ['Title'] )
# print dataframe.
int_title_df

,Title
0,Spain-int.txt
1,Royal Baby-int.txt
2,WNL The lightweight-int.txt
3,Arctic mapping-int.txt
4,WNL Mystery Shopper-int.txt
...,...
184,Liberia-int.txt
185,Meteorite-int.txt
186,Loterry-int.txt
187,WNL India's rich-int.txt


In [176]:
int_df1 = pd.concat([int_title_df,int_df], axis = 1)
int_df1

,Title,Text,Level
0,Spain-int.txt,Intermediate \nValdevaqueros is one of the las...,1
1,Royal Baby-int.txt,Intermediate \nThe Duchess of Cambridge gave b...,1
2,WNL The lightweight-int.txt,Intermediate \nCathal Redmond was swimming off...,1
3,Arctic mapping-int.txt,Intermediate \nIt has mapped the worlds highes...,1
4,WNL Mystery Shopper-int.txt,Intermediate \nThe customer next to you in the...,1
...,...,...,...
184,Liberia-int.txt,Intermediate \nEver since he was diagnosed HIV...,1
185,Meteorite-int.txt,Intermediate \nGalina Zaglumyonova was woken i...,1
186,Loterry-int.txt,"Intermediate \nAll six numbers match, so its t...",1
187,WNL India's rich-int.txt,"Intermediate \nAccording to a recent report, t...",1


In [177]:
int_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189 entries, 0 to 188
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    189 non-null    object
 1   Level   189 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 3.1+ KB


### Elementary List

In [178]:
ele_list = []
ele = '/Users/javm/Desktop/Flatiron4/Reading-Level-Classifier/Texts-SeparatedByReadingLevel/Ele-Txt'
for i in (os.listdir(ele)):
    with open(ele+'/'+i) as f:
        lines = f.read()
        ele_list.append(lines)
        
#print(ele_list)
    

In [179]:
ele_list_title = list(os.listdir(ele))

In [180]:
# initialize list of lists
data2 = ele_list
# Create the pandas DataFrame
ele_df = pd.DataFrame(data2, columns = ['Text'] )
# print dataframe.
ele_df['Level'] = 0
ele_df

,Text,Level
0,﻿SeaWorld’s profits fell by 84% and customers ...,0
1,﻿Imagine that you read a headline 'Fit in four...,0
2,﻿Robert Mysłajek stops. Between two paw prints...,0
3,﻿The Taliban sent a gunman to shoot Malala You...,0
4,﻿Governments in Europe dream of finding a magi...,0
...,...,...
184,"﻿Two scientists at Stanford University, in the...",0
185,﻿Barack Obama has told young people to reject ...,0
186,﻿When two people on a remote Pacific island sa...,0
187,﻿Google has made maps of the world’s highest m...,0


In [181]:
data33 = ele_list_title
# Create the pandas DataFrame
ele_title_df = pd.DataFrame(data33, columns = ['Title'] )
# print dataframe.
ele_title_df

,Title
0,WNL SeaWorld-ele.txt
1,Exercise-ele.txt
2,WNL On the trail-ele.txt
3,Malala-ele.txt
4,WNL Novel way-ele.txt
...,...
184,WNL Basic phone logs-ele.txt
185,WNL Barack Obama-ele.txt
186,WNL Castaway-ele.txt
187,WNL Arctic Ramadan-ele.txt


In [182]:
ele_df1 = pd.concat([ele_title_df,ele_df], axis = 1)
ele_df1

,Title,Text,Level
0,WNL SeaWorld-ele.txt,﻿SeaWorld’s profits fell by 84% and customers ...,0
1,Exercise-ele.txt,﻿Imagine that you read a headline 'Fit in four...,0
2,WNL On the trail-ele.txt,﻿Robert Mysłajek stops. Between two paw prints...,0
3,Malala-ele.txt,﻿The Taliban sent a gunman to shoot Malala You...,0
4,WNL Novel way-ele.txt,﻿Governments in Europe dream of finding a magi...,0
...,...,...,...
184,WNL Basic phone logs-ele.txt,"﻿Two scientists at Stanford University, in the...",0
185,WNL Barack Obama-ele.txt,﻿Barack Obama has told young people to reject ...,0
186,WNL Castaway-ele.txt,﻿When two people on a remote Pacific island sa...,0
187,WNL Arctic Ramadan-ele.txt,﻿Google has made maps of the world’s highest m...,0


In [183]:
ele_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189 entries, 0 to 188
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    189 non-null    object
 1   Level   189 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 3.1+ KB


### Combining Data Frames

In [184]:
df1 = adv_df1.merge(int_df1, how = 'outer')
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 378 entries, 0 to 377
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Title   378 non-null    object
 1   Text    378 non-null    object
 2   Level   378 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 11.8+ KB


In [215]:
df2 = df1.merge(ele_df1, how = 'outer')
df2['Title'].replace('-', ' ', regex=True, inplace = True)

In [217]:
new_feat_df['Title'].replace('-', ' ', regex=True, inplace = True)

In [223]:
df2.sort_values(by = 'Title', ascending = False)

,Title,Text,Level
342,climate change int.txt,Intermediate \nLow-income countries will conti...,1
536,climate change ele.txt,﻿Poorer countries will be most affected by cli...,0
55,climate change adv.txt,﻿Low-income countries will remain on the front...,2
197,Zero Hours int.txt,Intermediate \nMore than one million British w...,1
398,Zero Hours ele.txt,﻿More than one million British workers might b...,0
...,...,...,...
520,Amsterdam ele.txt,"﻿To tourists, Amsterdam still seems very liber...",0
35,Amsterdam adv.txt,"﻿Amsterdam still looks liberal to tourists, wh...",2
320,Amazon int.txt,"Intermediate\nWhen you see the word Amazon, wh...",1
523,Amazon ele.txt,"﻿When you see the word Amazon, what’s the firs...",0


In [214]:
print(df2.shape)
print(new_feat_df.shape)

(567, 3)
(567, 157)


In [226]:
df = df2.merge(new_feat_df, on = 'Title', how = 'outer')

In [241]:
df.dropna(inplace =True)


In [243]:
df.Text

0      ﻿It was not so much how hard people found the ...
1      ﻿Swedish prisons have long had a reputation ar...
2      A lonely old man living in a crater on the moo...
3      ﻿Facebook has lost millions of users per month...
4      ﻿Noise emanating from passing ships may distur...
                             ...                        
562    ﻿Two scientists at Stanford University, in the...
563    ﻿Barack Obama has told young people to reject ...
564    ﻿When two people on a remote Pacific island sa...
565    ﻿Google has made maps of the world’s highest m...
566    ﻿Our new international survey, including 33 co...
Name: Text, Length: 558, dtype: object

In [ ]:
feat_names = preprocess_pipeline[
    'countvec'].get_feature_names()

pd.DataFrame(X_tr_proc.toarray(), columns = feat_names)

In [117]:
df[text]

vec_text.shape 
df.shape
pd.concat()


NameError: name 'text' is not defined

## Preprocessing

In [27]:
# need to downlod punkt to access better tokenization rules
# word_tokenize won't work without it
#nltk.download('punkt') 

In [28]:
#corpus = [word_tokenize(doc) for doc in df['Text']]
#print(corpus[0:4])

In [29]:
#import itertools
#flattenedcorpus_tokens = pd.Series(list(itertools.chain(*corpus)))
#print(flattenedcorpus_tokens.shape)

In [30]:
#dictionary = pd.Series(flattenedcorpus_tokens.unique())
#print(len(dictionary))

In [31]:
#flattenedcorpus_tokens.value_counts()

In [32]:
#num_one_occurence = (flattenedcorpus_tokens.
                     #value_counts() < 2).sum()
#num_one_occurence

In [33]:
#dictionary[dictionary.str.isnumeric()]

### Removing Stop Words

In [34]:
# imports package with many stopword lists
from nltk.corpus import stopwords

# get common stop words in english that we'll remove during tokenization/text normalization
stop_words = stopwords.words('english')
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [35]:
#def first_step_normalizer(doc):
    # filters for alphabetic (no punctuation or numbers) and filters out stop words. 
    # lower cases all tokens
    #norm_text = [x.lower() for x in word_tokenize(doc) if ((x.isalpha()) & (x.lower() not in stop_words)) ]
    #return norm_text

In [36]:
#df['tok_norm'] = df['Text'].apply(first_step_normalizer)
#df.head(10)

In [37]:
#norm_toks_flattened = pd.Series(list(
    #itertools.chain(*df['tok_norm'])))
#new_dictionary = norm_toks_flattened.unique()
#print(len(new_dictionary))

In [38]:
#print(len(dictionary))

In [39]:
## Removed 5k features

In [40]:
#norm_toks_flattened.value_counts()

## Lemmatizing

In [41]:
from nltk import WordNetLemmatizer # lemmatizer using WordNet
from nltk.corpus import wordnet # imports WordNet
from nltk import pos_tag # nltk's native part of speech tagging

nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/javm/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /Users/javm/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [42]:
#fully_normalized_corpus = df['Text'].apply(process_doc)

In [43]:
#fully_normalized_corpus.head()

In [45]:
#flattened_fully_norm = pd.Series(list(itertools.chain(*fully_normalized_corpus)))
#len(flattened_fully_norm.unique())

In [46]:
## Original dictionary length
#print(len(dictionary))

In [47]:
#flattened_fully_norm.value_counts()

## Transforming

In [48]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

In [49]:
class TextPreprocessor(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        
        #define attributes to store if text preprocessing requires fitting from data
        pass
    
    def fit(self, data, y = 0):
        # this is where you would fit things like corpus specific stopwords
        # fit probable bigrams with bigram model in here
        
        # save as parameters of Text preprocessor
        
        return self
    
    def transform(self, data, y = 0):
        fully_normalized_corpus = data.apply(self.process_doc)
        
        return fully_normalized_corpus
        
    
    def process_doc(self, doc):

        #initialize lemmatizer
        wnl = WordNetLemmatizer()
        stop_words = stopwords.words('english')
        
        # helper function to change nltk's part of speech tagging to a wordnet format.
        def pos_tagger(nltk_tag):
            if nltk_tag.startswith('J'):
                return wordnet.ADJ
            elif nltk_tag.startswith('V'):
                return wordnet.VERB
            elif nltk_tag.startswith('N'):
                return wordnet.NOUN
            elif nltk_tag.startswith('R'):
                return wordnet.ADV
            else:         
                return None


        # remove stop words and punctuations, then lower case
        doc_norm = [tok.lower() for tok in word_tokenize(doc) if ((tok.isalpha()) & (tok not in stop_words)) ]

        #  POS detection on the result will be important in telling Wordnet's lemmatizer how to lemmatize

        # creates list of tuples with tokens and POS tags in wordnet format
        wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tag(doc_norm))) 
        doc_norm = [wnl.lemmatize(token, pos) for token, pos in wordnet_tagged if pos is not None]

        return " ".join(doc_norm)

In [50]:
X = df.drop(columns = ['Level'])
y = df['Level']

In [69]:
df

,Text,Level,AoA_Bird_Lem,AoA_Bristol_Lem,AoA_Cort_Lem,AoA_Kup,AoA_Kup_Lem,DISC_RefExprDefArtPerSen,DISC_RefExprDefArtPerWord,DISC_RefExprPerProPerWord,...,PROPORTION_INDEF_NP_REFCHAIN,PROPORTION_DEF_NP_REFCHAIN,PROPORTION_PERSONAL_PRONOUNS_REFCHAIN,PROPORTION_POSSESIVE_DETERMINERS_REFCHAIN,PROPORTION_DEMONSTRATIVE_DETERMINERS_REFCHAIN,PROPORTION_DEMONSTRATIVE_PRONOUNS_REFCHAIN,PROPORTION_REFLEXIVE_PRONOUNS_REFCHAIN,PROPORTION_PROPER_NOUNS_REFCHAIN,AVG_LEN_REFCHAIN,Level
0,﻿It was not so much how hard people found the ...,2,3.36,1.50,2.44,4.67,6.10,1.25,0.07,0.01,...,0.200000,0.485714,0.200000,0.0,0.0,0.0,0.000000,0.171429,2.500000,2
1,﻿Swedish prisons have long had a reputation ar...,2,3.65,1.11,2.43,4.56,5.76,0.84,0.06,0.02,...,0.115385,0.615385,0.230769,0.0,0.0,0.0,0.000000,0.461538,2.600000,0
2,A lonely old man living in a crater on the moo...,2,3.51,1.30,2.43,4.60,5.93,1.12,0.07,0.01,...,0.166667,0.466667,0.233333,0.0,0.0,0.0,0.000000,0.233333,2.727273,1
3,﻿Facebook has lost millions of users per month...,2,3.77,1.35,2.57,4.43,5.94,1.31,0.05,0.03,...,0.179487,0.461538,0.358974,0.0,0.0,0.0,0.000000,0.179487,3.545455,2
4,﻿Noise emanating from passing ships may distur...,2,3.82,1.17,2.74,4.36,5.51,1.13,0.05,0.04,...,0.208333,0.375000,0.416667,0.0,0.0,0.0,0.000000,0.166667,3.692308,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562,"﻿Two scientists at Stanford University, in the...",0,3.98,1.48,2.55,4.47,5.98,0.95,0.04,0.03,...,0.195122,0.268293,0.317073,0.0,0.0,0.0,0.024390,0.048780,3.727273,0
563,﻿Barack Obama has told young people to reject ...,0,4.06,1.40,2.52,4.30,5.98,1.46,0.05,0.02,...,0.170213,0.382979,0.255319,0.0,0.0,0.0,0.021277,0.021277,3.615385,1
564,﻿When two people on a remote Pacific island sa...,0,3.81,1.41,2.45,4.33,5.91,1.16,0.05,0.01,...,0.185185,0.629630,0.333333,0.0,0.0,0.0,0.000000,0.222222,2.700000,2
565,﻿Google has made maps of the world’s highest m...,0,4.16,1.28,2.68,3.99,5.40,0.96,0.05,0.01,...,0.320000,0.480000,0.320000,0.0,0.0,0.0,0.000000,0.200000,2.777778,0


In [245]:
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X['Text'], y, test_size = 0.3)

proc = TextPreprocessor()

# again this kind of splitting only becomes important if fitting of text transformers fits to statistics of the text corpus
transformed_train = proc.fit_transform(df['Text'])

In [246]:
transformed_train

0      much hard people find challenge far go avoid l...
1      prison long reputation world progressive count...
2      lonely old man living crater moon unlikely foc...
3      lose million user month big market independent...
4      emanate pass ship disturb animal killer whale ...
                             ...                        
562    scientist stanford university usa use metadata...
563    obama tell young people reject pessimism meet ...
564    people remote pacific island saw small boat wa...
565    make map world high mountain ocean floor amazo...
566    new international survey include country show ...
Name: Text, Length: 558, dtype: object

In [64]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [618]:
#prc_steps = [('countvec', CountVectorizer(min_df =.03, max_df = .5))]
#preprocess_pipeline = Pipeline(prc_steps)

In [56]:
numeric_features = df.drop( columns = ['Text','Level'])
numeric_transformer = Pipeline(    
    steps=[("scaler", StandardScaler())])

text_features = df['Text']
text_transformer = Pipeline(steps=[
    ('countvec', CountVectorizer(min_df =.03, max_df = .5))])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("text", text_transformer, text_features),])

In [65]:
clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LogisticRegression(random_state=42))]
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))

ValueError: No valid specification of the columns. Only a scalar, list or slice of all integers or all strings, or boolean mask is allowed

## StandardScaler

In [630]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_num_scaled = scaler.fit_transform(X_train_num)

X_test_num_scaled = scaler.transform(X_test_num)

In [638]:
X_train_num_scaled_df = pd.DataFrame(X_train_num_scaled, columns = X_train_num.columns ) 
X_test_num_scaled_df = pd.DataFrame(X_test_num_scaled, columns = X_test_num.columns ) 

In [650]:
train_df = pd.concat([X_tr_proc_df, X_train_num_scaled_df], axis = 1 )

test_df = pd.concat([X_test_proc_df, X_test_num_scaled_df], axis = 1 )

In [251]:
feat_names = preprocess_pipeline[
    'countvec'].get_feature_names()

pd.DataFrame(X_tr_proc2.toarray(), columns = feat_names)

NotFittedError: Vocabulary not fitted or provided

In [658]:
train_df

,0,AoA_Bird_Lem,AoA_Bristol_Lem,AoA_Cort_Lem,AoA_Kup,AoA_Kup_Lem,DISC_RefExprDefArtPerSen,DISC_RefExprDefArtPerWord,DISC_RefExprPerProPerWord,DISC_RefExprPerPronounsPerSen,...,AVG_NUM_WORD_PER_ENTITY,PROPORTION_INDEF_NP_REFCHAIN,PROPORTION_DEF_NP_REFCHAIN,PROPORTION_PERSONAL_PRONOUNS_REFCHAIN,PROPORTION_POSSESIVE_DETERMINERS_REFCHAIN,PROPORTION_DEMONSTRATIVE_DETERMINERS_REFCHAIN,PROPORTION_DEMONSTRATIVE_PRONOUNS_REFCHAIN,PROPORTION_REFLEXIVE_PRONOUNS_REFCHAIN,PROPORTION_PROPER_NOUNS_REFCHAIN,AVG_LEN_REFCHAIN
0,"(0, 735)\t5\n (0, 10)\t5\n (0, 695)\t1\n ...",0.637511,1.560606,-1.627561,2.429856,1.127638,-0.553891,-0.355282,-0.699945,-0.555624,...,0.0,0.709873,0.093698,-0.695000,0.0,-0.416693,0.0,-0.455915,-1.224623,-0.658360
1,"(0, 204)\t2\n (0, 278)\t1\n (0, 550)\t1\n ...",0.637511,-1.420236,1.447045,-1.079433,-1.458029,-0.387598,-0.355282,1.345651,1.431411,...,0.0,-1.358099,-0.417934,1.349245,0.0,-0.416693,0.0,0.552395,-0.268286,3.661622
2,"(0, 176)\t2\n (0, 471)\t1\n (0, 137)\t1\n ...",-0.390179,-0.202709,0.348971,-0.720528,0.804430,1.370361,1.410724,-0.699945,-0.407338,...,0.0,0.849582,0.842609,-0.555505,0.0,-0.416693,0.0,-0.455915,0.497821,-0.343891
3,"(0, 238)\t4\n (0, 254)\t4\n (0, 639)\t1\n ...",0.101325,-0.790481,0.898008,-0.561015,-1.425708,0.728943,0.233386,-0.699945,-0.348023,...,0.0,-1.041200,1.929701,-0.576249,0.0,-0.416693,0.0,-0.455915,1.361237,-0.163362
4,"(0, 306)\t1\n (0, 542)\t1\n (0, 378)\t1\n ...",2.335432,-0.706513,-1.188332,0.236551,0.513542,-1.337846,-1.532620,0.834252,0.423065,...,0.0,0.853324,-0.080879,0.717092,0.0,1.229984,0.0,-0.455915,-0.648467,-0.580713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,"(0, 424)\t1\n (0, 388)\t1\n (0, 204)\t1\n ...",0.146007,-0.496595,0.788201,0.715090,-1.005537,0.990262,1.410724,-0.188546,-0.140423,...,0.0,1.155091,0.749940,-0.826882,0.0,-0.416693,0.0,-0.455915,1.245547,-0.232991
392,"(0, 254)\t2\n (0, 278)\t3\n (0, 639)\t1\n ...",0.369418,-1.126350,1.117623,-0.002719,-0.714650,-1.337846,-1.532620,1.857051,1.757641,...,0.0,-0.695950,-1.564785,1.475375,0.0,-0.416693,0.0,-0.455915,-1.173571,-0.425420
393,"(0, 363)\t1\n (0, 471)\t1\n (0, 137)\t1\n ...",0.235371,0.007209,-0.858910,0.435942,-0.068233,-0.197548,-0.355282,-0.188546,-0.051451,...,0.0,-0.954162,-0.080879,0.868749,0.0,1.229984,0.0,-0.455915,-0.156181,-0.580713
394,"(0, 542)\t1\n (0, 471)\t1\n (0, 380)\t1\n ...",0.860921,0.636965,-0.913814,-0.002719,0.448901,-0.031255,0.233386,-0.188546,-0.140423,...,0.0,-0.850877,-0.006060,0.262122,0.0,2.217991,0.0,-0.455915,-0.320276,0.302518


In [623]:
bow_mat = pd.DataFrame(X_tr_proc.toarray(), columns = feat_names)
bow_mat['Level'] = y_train
bow_mat

ValueError: Wrong number of items passed 2, placement implies 1

In [255]:
class2_bow_mat = bow_mat[bow_mat['Level'] == 2].drop(columns = ['Level'])

# class 1 token probabilities:
N_tok_2 = class2_bow_mat.sum(axis = 0) 
N_2 =  class2_bow_mat.values.sum()

# get probabilities for each token: class 1
proba_c2 = N_tok_2/N_2

proba_c2.sort_values(ascending = False)[0:10]

NameError: name 'bow_mat' is not defined

In [ ]:
class1_bow_mat = bow_mat[bow_mat['Level'] == 1].drop(columns = ['Level'])

# class 1 token probabilities:
N_tok_1 = class1_bow_mat.sum(axis = 0) 
N_1 =  class1_bow_mat.values.sum()

# get probabilities for each token: class 1
proba_c1 = N_tok_1/N_1

proba_c1.sort_values(ascending = False)[0:10]

In [ ]:
class0_bow_mat = bow_mat[bow_mat['Level'] == 0].drop(columns = ['Level'])

# class 1 token probabilities:
N_tok_0 = class0_bow_mat.sum(axis = 0)
N_0 =  class0_bow_mat.values.sum() 

# get probabilities for each token: class 0
proba_c0 = N_tok_0/N_0

proba_c0.sort_values(ascending = False)[0:10]

### Append Multinomial Naive Bayes Classifier to pipeline.

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
from copy import deepcopy
mod_pipe = deepcopy(preprocess_pipeline)
mod_pipe.steps.append(('multinb', MultinomialNB()))
mod_pipe.steps

In [ ]:
mod_pipe.fit(transformed_train, y_train)

In [ ]:
transformed_test = proc.transform(X_test)

y_pred = mod_pipe.predict(transformed_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
import sklearn
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.datasets import load_breast_cancer 
from sklearn.svm import SVC 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split 

# LOOK HERE

In [ ]:
# defining parameter range 
param_grid = {'C': [0.1, 1, 10, 100],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              #'gamma':['scale', 'auto'],
              'kernel': ['linear']}  
   
grid = GridSearchCV(SVC(random_state = 42), param_grid, refit = True, verbose = 3,n_jobs=-1) 
   
# fitting the model for grid search 
grid.fit(X_tr_proc, y_train) 
 
# print best parameter after tuning 
print(grid.best_params_) 
grid_predictions = grid.predict(X_test_proc) 
   
# print classification report 
print(classification_report(y_test, grid_predictions)) 

### TFID Vectorizer

In [248]:
X_tr_proc2 = preprocess_pipeline2.fit_transform(transformed_train)

In [247]:
prc_steps2 = [('tfid', TfidfVectorizer(min_df = 0.05, max_df = 0.95))]
preprocess_pipeline2 = Pipeline(prc_steps2)

In [254]:
feat_names = preprocess_pipeline2[
    'tfid'].get_feature_names()

word_vec = pd.DataFrame(X_tr_proc2.toarray(), columns = feat_names)
pd.concat([df, word_vec],1).shape

(567, 543)

In [ ]:
X_test_proc2 = preprocess_pipeline2.transform(transformed_test)

In [ ]:
# defining parameter range 
param_grid = {'C': [0.1, 1, 10, 100],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              #'gamma':['scale', 'auto'],
              'kernel': ['linear']}  
   
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3,n_jobs=-1) 
   
# fitting the model for grid search 
grid.fit(X_tr_proc2, y_train) 
 
# print best parameter after tuning 
print(grid.best_params_) 
grid_predictions2 = grid.predict(X_test_proc2) 
   
# print classification report 
print(classification_report(y_test, grid_predictions2))

### NB

In [ ]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB
 
# GaussianNB is used in Binomial Classification
# MultinomialNB is used in multi-class classification
#clf = GaussianNB()
clf = MultinomialNB()
 
# Printing all the parameters of Naive Bayes
print(clf)
 
NB=clf.fit(X_tr_proc,y_train)
prediction=NB.predict(X_test_proc)
 
# Measuring accuracy on Testing Data
from sklearn import metrics
print(metrics.classification_report(y_test, prediction))
print(metrics.confusion_matrix(y_test, prediction))
 
# Printing the Overall Accuracy of the model
F1_Score=metrics.f1_score(y_test, prediction, average='weighted')
print('Accuracy of the model on Testing Sample Data:', round(F1_Score,2))
 
# Importing cross validation function from sklearn
from sklearn.model_selection import cross_val_score
 
# Running 10-Fold Cross validation on a given algorithm
# Passing full data X and y because the K-fold will split the data and automatically choose train/test
Accuracy_Values=cross_val_score(NB, X , y, cv=5, scoring='f1_weighted')
print('\nAccuracy values for 5-fold Cross Validation:\n',Accuracy_Values)
print('\nFinal Average Accuracy of the model:', round(Accuracy_Values.mean(),2))

In [ ]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB
 
# GaussianNB is used in Binomial Classification
# MultinomialNB is used in multi-class classification
#clf = GaussianNB()
clf = ComplementNB()
 
# Printing all the parameters of Naive Bayes
print(clf)
 
NB=clf.fit(X_tr_proc2,y_train)
prediction=NB.predict(X_test_proc2)
 
# Measuring accuracy on Testing Data
from sklearn import metrics
print(metrics.classification_report(y_test, prediction))
print(metrics.confusion_matrix(y_test, prediction))
 
# Printing the Overall Accuracy of the model
F1_Score=metrics.f1_score(y_test, prediction, average='weighted')
print('Accuracy of the model on Testing Sample Data:', round(F1_Score,2))
 
# Importing cross validation function from sklearn
from sklearn.model_selection import cross_val_score
 
# Running 10-Fold Cross validation on a given algorithm
# Passing full data X and y because the K-fold will split the data and automatically choose train/test
Accuracy_Values=cross_val_score(NB, X , y, cv=5, scoring='f1_weighted')
print('\nAccuracy values for 5-fold Cross Validation:\n',Accuracy_Values)
print('\nFinal Average Accuracy of the model:', round(Accuracy_Values.mean(),2))